In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from itertools import product
from collections import defaultdict

In [4]:
from sklearn.metrics import roc_auc_score

In [5]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [6]:
series = pd.read_csv('train.csv',index_col=['sequence','subject','step'])
series.head()

sensor_00  sensor_01  sensor_02  sensor_03  sensor_04  \
sequence subject step                                                          
0        47      0     -0.196291   0.112395        1.0   0.329204  -1.004660   
                 1     -0.447450   0.134454        1.0  -0.658407   0.162495   
                 2      0.326893  -0.694328        1.0   0.330088   0.473678   
                 3      0.523184   0.751050        1.0   0.976991  -0.563287   
                 4      0.272025   1.074580        1.0  -0.136283   0.398579   

                       sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
sequence subject step                                                          
0        47      0     -0.131638  -0.127505   0.368702       -0.1  -0.963873   
                 1      0.340314  -0.209472  -0.867176        0.2  -0.301301   
                 2      1.280479  -0.094718   0.535878        1.4   1.002168   
                 3     -0.720269   0.793260   0.951145       -0.3  -0.995665   
                 4      0.044877   0.560109  -0.541985       -0.9   1.055636   

                       sensor_10  sensor_11  sensor_12  
sequence subject step                                   
0        47      0     -0.985069   0.531893   4.751492  
                 1      0.082733  -0.231481   0.454390  
                 2      0.449221  -0.586420  -4.736147  
                 3     -0.434290   1.344650   0.429241  
                 4      0.812631   0.123457  -0.223359

In [7]:
labels = pd.read_csv('train_labels.csv', index_col=['sequence'])
labels.head()

,state
sequence,
0,0
1,1
2,1
3,1
4,1


In [12]:
test = pd.read_csv('test.csv',index_col=['sequence','subject','step'])

In [13]:
test.head()

sensor_00  sensor_01  sensor_02  sensor_03  sensor_04  \
sequence subject step                                                          
25968    684     0      2.427357  19.639706        1.0  -1.466372  -1.289973   
                 1     -4.950541 -21.747899        1.0   0.983186  -0.569053   
                 2      1.136012 -10.756303        1.0   1.016814   0.964157   
                 3      0.806028   6.504202        1.0  -0.179646   0.969221   
                 4      1.288253   5.552521        1.0  -0.493805  -1.036124   

                       sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
sequence subject step                                                          
25968    684     0     -4.207928   2.486339  -2.493893        8.0  -1.123555   
                 1      1.845924  -3.887978   1.727481       -2.9   0.395231   
                 2      2.454749   0.312386   1.154198       -5.6   1.114162   
                 3     -1.035153  -0.457195   0.254962       -2.7  -0.588873   
                 4     -1.126402   2.008197  -0.730534        0.0   0.899566   

                       sensor_10  sensor_11  sensor_12  
sequence subject step                                   
25968    684     0     -1.673048  10.980453   0.419011  
                 1     -0.882233  -1.871399  -0.008525  
                 2      1.525273 -11.584362   0.139812  
                 3      0.608761  -4.241770  -0.462916  
                 4     -1.259615  -0.472222  -0.121483

In [9]:
series.index.get_level_values(1).unique()

Int64Index([ 47,  66, 542, 437, 510, 246, 520,  78, 526,   4,
            ...
            506,  45,  39, 181, 490, 616, 486, 202,  73, 112],
           dtype='int64', name='subject', length=672)

In [14]:
test.index.get_level_values(1).unique()

Int64Index([684, 935, 924, 769, 764, 988, 790, 965, 887, 921,
            ...
            958, 943, 786, 839, 975, 720, 802, 832, 738, 714],
           dtype='int64', name='subject', length=319)

In [32]:
tmp = sorted(series.index.get_level_values(1).unique().to_list())

In [33]:
tmp

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [35]:
for subject in sorted(test.index.get_level_values(1).unique().to_list()):
    print(subject)

672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921


The objects are different between the dataset.

In [39]:
data_all = pd.concat([series, test])
data_all.shape

(2291160, 13)

In [43]:
mean = data_all.mean(axis=0)
mean

sensor_00    0.000363
sensor_01   -0.001039
sensor_02   -0.217954
sensor_03   -0.002250
sensor_04   -0.001293
sensor_05   -0.001735
sensor_06   -0.000827
sensor_07    0.000242
sensor_08   -0.000019
sensor_09    0.001277
sensor_10   -0.000139
sensor_11   -0.003435
sensor_12   -0.008596
dtype: float64

In [46]:
std = data_all.std(axis=0)
std

sensor_00     2.747429
sensor_01     4.466203
sensor_02     2.310527
sensor_03     4.261084
sensor_04     1.749201
sensor_05     1.601893
sensor_06     3.370294
sensor_07     3.099620
sensor_08     4.663098
sensor_09     2.659309
sensor_10     1.980841
sensor_11     4.490679
sensor_12    39.669326
dtype: float64

In [45]:
series

sensor_00  sensor_01  sensor_02  sensor_03  sensor_04  \
sequence subject step                                                          
0        47      0     -0.196291   0.112395   1.000000   0.329204  -1.004660   
                 1     -0.447450   0.134454   1.000000  -0.658407   0.162495   
                 2      0.326893  -0.694328   1.000000   0.330088   0.473678   
                 3      0.523184   0.751050   1.000000   0.976991  -0.563287   
                 4      0.272025   1.074580   1.000000  -0.136283   0.398579   
...                          ...        ...        ...        ...        ...   
25967    327     55    -0.282844  -1.217437  -1.666153   0.586726  -0.930698   
                 56     0.130603   0.349790  -1.666153  -0.324779   0.775324   
                 57    -0.579598   0.429622  -1.666153   0.319469   0.308861   
                 58     1.278980   1.711134  -1.522820   0.802655  -0.460541   
                 59    -1.136012  -3.702731  -1.332820  -0.766372  -0.430027   

                       sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
sequence subject step                                                          
0        47      0     -0.131638  -0.127505   0.368702       -0.1  -0.963873   
                 1      0.340314  -0.209472  -0.867176        0.2  -0.301301   
                 2      1.280479  -0.094718   0.535878        1.4   1.002168   
                 3     -0.720269   0.793260   0.951145       -0.3  -0.995665   
                 4      0.044877   0.560109  -0.541985       -0.9   1.055636   
...                          ...        ...        ...        ...        ...   
25967    327     55    -0.451010  -0.651184   0.368702        0.4   0.008671   
                 56    -0.332835   0.099271   0.122137       -0.2   0.644509   
                 57     0.282723  -0.512750   0.012214       -1.6  -0.424133   
                 58    -0.055348   2.405282   0.043511        1.9   0.283960   
                 59    -0.091997  -2.512750  -0.022901       -1.1  -0.653902   

                       sensor_10  sensor_11  sensor_12  
sequence subject step                                   
0        47      0     -0.985069   0.531893   4.751492  
                 1      0.082733  -0.231481   0.454390  
                 2      0.449221  -0.586420  -4.736147  
                 3     -0.434290   1.344650   0.429241  
                 4      0.812631   0.123457  -0.223359  
...                          ...        ...        ...  
25967    327     55    -0.723536  -0.353909  -0.914749  
                 56     0.691407  -0.613169  -0.515772  
                 57     0.716855   1.628601   0.928389  
                 58    -0.914914   0.364198   0.211424  
                 59    -0.418516  -1.453704  -1.561381  

[1558080 rows x 13 columns]

In [47]:
series / std

sensor_00  sensor_01  sensor_02  sensor_03  sensor_04  \
sequence subject step                                                          
0        47      0     -0.071445   0.025166   0.432802   0.077258  -0.574354   
                 1     -0.162861   0.030105   0.432802  -0.154516   0.092897   
                 2      0.118982  -0.155463   0.432802   0.077466   0.270797   
                 3      0.190427   0.168163   0.432802   0.229282  -0.322025   
                 4      0.099011   0.240603   0.432802  -0.031983   0.227863   
...                          ...        ...        ...        ...        ...   
25967    327     55    -0.102949  -0.272589  -0.721114   0.137694  -0.532070   
                 56     0.047536   0.078319  -0.721114  -0.076220   0.443245   
                 57    -0.210960   0.096194  -0.721114   0.074974   0.176573   
                 58     0.465519   0.383130  -0.659079   0.188369  -0.263287   
                 59    -0.413482  -0.829056  -0.576847  -0.179854  -0.245842   

                       sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
sequence subject step                                                          
0        47      0     -0.082177  -0.037832   0.118951  -0.021445  -0.362452   
                 1      0.212445  -0.062152  -0.279768   0.042890  -0.113300   
                 2      0.799354  -0.028104   0.172885   0.300230   0.376853   
                 3     -0.449636   0.235368   0.306859  -0.064335  -0.374407   
                 4      0.028015   0.166190  -0.174855  -0.193005   0.396959   
...                          ...        ...        ...        ...        ...   
25967    327     55    -0.281548  -0.193213   0.118951   0.085780   0.003260   
                 56    -0.207776   0.029455   0.039404  -0.042890   0.242359   
                 57     0.176493  -0.152138   0.003940  -0.343120  -0.159490   
                 58    -0.034551   0.713671   0.014038   0.407454   0.106779   
                 59    -0.057430  -0.745558  -0.007388  -0.235895  -0.245892   

                       sensor_10  sensor_11  sensor_12  
sequence subject step                                   
0        47      0     -0.497298   0.118444   0.119777  
                 1      0.041767  -0.051547   0.011454  
                 2      0.226783  -0.130586  -0.119391  
                 3     -0.219245   0.299431   0.010820  
                 4      0.410245   0.027492  -0.005631  
...                          ...        ...        ...  
25967    327     55    -0.365267  -0.078810  -0.023059  
                 56     0.349047  -0.136543  -0.013002  
                 57     0.361894   0.362662   0.023403  
                 58    -0.461882   0.081101   0.005330  
                 59    -0.211282  -0.323716  -0.039360  

[1558080 rows x 13 columns]

In [49]:
series = (series - mean) / std
series.head()

sensor_00  sensor_01  sensor_02  sensor_03  sensor_04  \
sequence subject step                                                          
0        47      0     -0.026185   0.005919   0.322475   0.018783  -0.327190   
                 1     -0.059458   0.007025   0.322475  -0.035610   0.054270   
                 2      0.043126  -0.034524   0.322475   0.018832   0.155974   
                 3      0.069130   0.037937   0.322475   0.054460  -0.182937   
                 4      0.035857   0.054156   0.322475  -0.006854   0.131429   

                       sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
sequence subject step                                                          
0        47      0     -0.049540  -0.010907   0.038273  -0.004594  -0.136957   
                 1      0.134381  -0.018123  -0.090362   0.009203  -0.043266   
                 2      0.500765  -0.008020   0.055673   0.064389   0.141050   
                 3     -0.278931   0.070154   0.098896  -0.013792  -0.141452   
                 4      0.019248   0.049628  -0.056515  -0.041385   0.148610   

                       sensor_10  sensor_11  sensor_12  
sequence subject step                                   
0        47      0     -0.250948   0.027311   0.003242  
                 1      0.021191  -0.010543   0.000511  
                 2      0.114594  -0.028144  -0.002787  
                 3     -0.110577   0.067614   0.000495  
                 4      0.207213   0.007057   0.000080

### do a brute force xgboost

In [53]:
seqs = series.index.get_level_values(0).unique()

In [51]:
X = np.zeros((len(seqs),60*13),dtype=np.float16)

In [52]:
for idx, seq in enumerate(seqs):
    if (idx % 100 == 0):  print(idx)
    X[idx] = series.loc[seq].values.reshape(1,-1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [55]:
X.shape

(25968, 780)

In [56]:
y = np.zeros(len(X), dtype = np.int0)

In [57]:
for idx,seq in enumerate(seqs):
    y[idx] = labels.loc[seq].values[0]

In [58]:
def gridsearch_xgboost(params, X, y, obj="binary:logistic"):
    ''' the params include both the fit and the booster params '''
    
    param_names, param_vals = zip(*list(params.items()))
    id_booster, id_fit = [], []
    for idx, name in enumerate(param_names):
        if (name=='early_stopping_rounds'):
            id_fit.append(idx)
        else:
            id_booster.append(idx)
    
    best_test_score = 999.0
    for val_combination in product(*param_vals):
        param_fit, param_booster = {}, {}
        param_fit['eval_metric'] = 'auc'
        for idx in id_fit:
            param_fit[param_names[idx]] = val_combination[idx]
        for idx in id_booster:
            param_booster[param_names[idx]] = val_combination[idx]
        print(f"param_fit: {param_fit}")
        print(f"param_booster: {param_booster}")
        
        clf = XGBClassifier(**param_booster, objective=obj, use_label_encoder=False)
        result = defaultdict(list)
        for train_idx, test_idx in kf.split(X):
            X_train, y_train = X[train_idx], y[train_idx]
            X_test, y_test = X[test_idx], y[test_idx]
            param_fit['eval_set'] = [(X_test, y_test)]            
            clf.fit(X_train, y_train, **param_fit, verbose=True)
            if hasattr(clf, 'best_ntree_limit'):
                result['ntree'].append(clf.best_ntree_limit)
            else:
                result['ntree'].append(clf.n_estimators)            
            result['score'].append(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))
            print(f'stop at {result["ntree"][-1]}, score={result["score"][-1]}')

        mean_test_score = np.mean(result['score'])
        print(f"mean_test_score={mean_test_score}")
        if mean_test_score > best_test_score:
            best_test_score = mean_test_score
            best_param_booster = param_booster
            best_param_booster['n_estimators'] = np.int32(np.mean(result['ntree']))
    
    return best_param_booster

In [59]:
params = {'gamma': [0],
          'max_depth': [6],
          'min_child_weight': [0.0],
          'n_estimators': [10000],
          'early_stopping_rounds': [50],
          'learning_rate': [0.3]}

In [60]:
best_param_booster = gridsearch_xgboost(params, X, y)

param_fit: {'eval_metric': 'auc', 'early_stopping_rounds': 50}
param_booster: {'gamma': 0, 'max_depth': 6, 'min_child_weight': 0.0, 'n_estimators': 10000, 'learning_rate': 0.3}
[0]	validation_0-auc:0.62888
[1]	validation_0-auc:0.65814
[2]	validation_0-auc:0.66803
[3]	validation_0-auc:0.66765
[4]	validation_0-auc:0.67249
[5]	validation_0-auc:0.67109
[6]	validation_0-auc:0.67251
[7]	validation_0-auc:0.67378
[8]	validation_0-auc:0.67423
[9]	validation_0-auc:0.67976
[10]	validation_0-auc:0.68673
[11]	validation_0-auc:0.68773
[12]	validation_0-auc:0.69004
[13]	validation_0-auc:0.68841
[14]	validation_0-auc:0.69167
[15]	validation_0-auc:0.69350
[16]	validation_0-auc:0.69456
[17]	validation_0-auc:0.69687
[18]	validation_0-auc:0.70070
[19]	validation_0-auc:0.70087
[20]	validation_0-auc:0.70149
[21]	validation_0-auc:0.70283
[22]	validation_0-auc:0.70286
[23]	validation_0-auc:0.70534
[24]	validation_0-auc:0.70658
[25]	validation_0-auc:0.70593
[26]	validation_0-auc:0.70569
[27]	validation_0-auc:0

[263]	validation_0-auc:0.74454
[264]	validation_0-auc:0.74437
[265]	validation_0-auc:0.74410
[266]	validation_0-auc:0.74397
[267]	validation_0-auc:0.74449
[268]	validation_0-auc:0.74431
[269]	validation_0-auc:0.74475
[270]	validation_0-auc:0.74475
[271]	validation_0-auc:0.74515
[272]	validation_0-auc:0.74537
[273]	validation_0-auc:0.74568
[274]	validation_0-auc:0.74560
[275]	validation_0-auc:0.74545
[276]	validation_0-auc:0.74553
[277]	validation_0-auc:0.74540
[278]	validation_0-auc:0.74521
[279]	validation_0-auc:0.74496
[280]	validation_0-auc:0.74503
[281]	validation_0-auc:0.74494
[282]	validation_0-auc:0.74509
[283]	validation_0-auc:0.74478
[284]	validation_0-auc:0.74535
[285]	validation_0-auc:0.74555
[286]	validation_0-auc:0.74571
[287]	validation_0-auc:0.74596
[288]	validation_0-auc:0.74606
[289]	validation_0-auc:0.74634
[290]	validation_0-auc:0.74651
[291]	validation_0-auc:0.74634
[292]	validation_0-auc:0.74632
[293]	validation_0-auc:0.74631
[294]	validation_0-auc:0.74684
[295]	va

[528]	validation_0-auc:0.76504
[529]	validation_0-auc:0.76512
[530]	validation_0-auc:0.76511
[531]	validation_0-auc:0.76495
[532]	validation_0-auc:0.76495
[533]	validation_0-auc:0.76498
[534]	validation_0-auc:0.76491
[535]	validation_0-auc:0.76506
[536]	validation_0-auc:0.76513
[537]	validation_0-auc:0.76517
[538]	validation_0-auc:0.76538
[539]	validation_0-auc:0.76538
[540]	validation_0-auc:0.76543
[541]	validation_0-auc:0.76540
[542]	validation_0-auc:0.76553
[543]	validation_0-auc:0.76558
[544]	validation_0-auc:0.76553
[545]	validation_0-auc:0.76562
[546]	validation_0-auc:0.76636
[547]	validation_0-auc:0.76643
[548]	validation_0-auc:0.76652
[549]	validation_0-auc:0.76651
[550]	validation_0-auc:0.76641
[551]	validation_0-auc:0.76720
[552]	validation_0-auc:0.76747
[553]	validation_0-auc:0.76804
[554]	validation_0-auc:0.76798
[555]	validation_0-auc:0.76784
[556]	validation_0-auc:0.76783
[557]	validation_0-auc:0.76793
[558]	validation_0-auc:0.76788
[559]	validation_0-auc:0.76785
[560]	va

[113]	validation_0-auc:0.72480
[114]	validation_0-auc:0.72455
[115]	validation_0-auc:0.72481
[116]	validation_0-auc:0.72530
[117]	validation_0-auc:0.72583
[118]	validation_0-auc:0.72672
[119]	validation_0-auc:0.72742
[120]	validation_0-auc:0.72806
[121]	validation_0-auc:0.72786
[122]	validation_0-auc:0.72777
[123]	validation_0-auc:0.72712
[124]	validation_0-auc:0.72744
[125]	validation_0-auc:0.72687
[126]	validation_0-auc:0.72637
[127]	validation_0-auc:0.72662
[128]	validation_0-auc:0.72671
[129]	validation_0-auc:0.72709
[130]	validation_0-auc:0.72715
[131]	validation_0-auc:0.72706
[132]	validation_0-auc:0.72830
[133]	validation_0-auc:0.73222
[134]	validation_0-auc:0.73198
[135]	validation_0-auc:0.73191
[136]	validation_0-auc:0.73170
[137]	validation_0-auc:0.73196
[138]	validation_0-auc:0.73264
[139]	validation_0-auc:0.73254
[140]	validation_0-auc:0.73238
[141]	validation_0-auc:0.73237
[142]	validation_0-auc:0.73266
[143]	validation_0-auc:0.73332
[144]	validation_0-auc:0.73309
[145]	va

[156]	validation_0-auc:0.72454
[157]	validation_0-auc:0.72483
[158]	validation_0-auc:0.72498
[159]	validation_0-auc:0.72482
[160]	validation_0-auc:0.72460
[161]	validation_0-auc:0.72496
[162]	validation_0-auc:0.72486
[163]	validation_0-auc:0.72499
[164]	validation_0-auc:0.72682
[165]	validation_0-auc:0.72698
[166]	validation_0-auc:0.72710
[167]	validation_0-auc:0.72783
[168]	validation_0-auc:0.72880
[169]	validation_0-auc:0.72933
[170]	validation_0-auc:0.72893
[171]	validation_0-auc:0.72915
[172]	validation_0-auc:0.72998
[173]	validation_0-auc:0.73011
[174]	validation_0-auc:0.73044
[175]	validation_0-auc:0.73035
[176]	validation_0-auc:0.73232
[177]	validation_0-auc:0.73278
[178]	validation_0-auc:0.73233
[179]	validation_0-auc:0.73244
[180]	validation_0-auc:0.73248
[181]	validation_0-auc:0.73262
[182]	validation_0-auc:0.73257
[183]	validation_0-auc:0.73292
[184]	validation_0-auc:0.73302
[185]	validation_0-auc:0.73283
[186]	validation_0-auc:0.73323
[187]	validation_0-auc:0.73588
[188]	va

...

[267]	validation_0-auc:0.74268
[268]	validation_0-auc:0.74284
[269]	validation_0-auc:0.74261
[270]	validation_0-auc:0.74246
[271]	validation_0-auc:0.74253
[272]	validation_0-auc:0.74266
[273]	validation_0-auc:0.74282
[274]	validation_0-auc:0.74286
[275]	validation_0-auc:0.74300
[276]	validation_0-auc:0.74289
[277]	validation_0-auc:0.74262
[278]	validation_0-auc:0.74285
[279]	validation_0-auc:0.74304
[280]	validation_0-auc:0.74332
[281]	validation_0-auc:0.74309
[282]	validation_0-auc:0.74314
[283]	validation_0-auc:0.74548
[284]	validation_0-auc:0.74584
[285]	validation_0-auc:0.74650
[286]	validation_0-auc:0.74657
[287]	validation_0-auc:0.74608
[288]	validation_0-auc:0.74757
[289]	validation_0-auc:0.74906
[290]	validation_0-auc:0.74894
[291]	validation_0-auc:0.74891
[292]	validation_0-auc:0.74894
[293]	validation_0-auc:0.74874
[294]	validation_0-auc:0.74824
[295]	validation_0-auc:0.74784
[296]	validation_0-auc:0.74777
[297]	validation_0-auc:0.74760
[298]	validation_0-auc:0.74771
[299]	va

[532]	validation_0-auc:0.76231
[533]	validation_0-auc:0.76221
[534]	validation_0-auc:0.76230
[535]	validation_0-auc:0.76228
[536]	validation_0-auc:0.76228
[537]	validation_0-auc:0.76374
[538]	validation_0-auc:0.76357
[539]	validation_0-auc:0.76358
[540]	validation_0-auc:0.76382
[541]	validation_0-auc:0.76372
[542]	validation_0-auc:0.76393
[543]	validation_0-auc:0.76395
[544]	validation_0-auc:0.76387
[545]	validation_0-auc:0.76374
[546]	validation_0-auc:0.76372
[547]	validation_0-auc:0.76388
[548]	validation_0-auc:0.76388
[549]	validation_0-auc:0.76393
[550]	validation_0-auc:0.76401
[551]	validation_0-auc:0.76392
[552]	validation_0-auc:0.76405
[553]	validation_0-auc:0.76409
[554]	validation_0-auc:0.76415
[555]	validation_0-auc:0.76416
[556]	validation_0-auc:0.76412
[557]	validation_0-auc:0.76415
[558]	validation_0-auc:0.76426
[559]	validation_0-auc:0.76412
[560]	validation_0-auc:0.76398
[561]	validation_0-auc:0.76399
[562]	validation_0-auc:0.76385
[563]	validation_0-auc:0.76381
[564]	va

[797]	validation_0-auc:0.76750
[798]	validation_0-auc:0.76751
[799]	validation_0-auc:0.76755
[800]	validation_0-auc:0.76755
[801]	validation_0-auc:0.76754
[802]	validation_0-auc:0.76763
[803]	validation_0-auc:0.76757
[804]	validation_0-auc:0.76764
[805]	validation_0-auc:0.76768
[806]	validation_0-auc:0.76763
[807]	validation_0-auc:0.76763
[808]	validation_0-auc:0.76760
[809]	validation_0-auc:0.76760
[810]	validation_0-auc:0.76758
[811]	validation_0-auc:0.76761
[812]	validation_0-auc:0.76765
[813]	validation_0-auc:0.76771
[814]	validation_0-auc:0.76767
[815]	validation_0-auc:0.76768
[816]	validation_0-auc:0.76765
[817]	validation_0-auc:0.76766
[818]	validation_0-auc:0.76766
[819]	validation_0-auc:0.76770
[820]	validation_0-auc:0.76762
[821]	validation_0-auc:0.76838
[822]	validation_0-auc:0.76849
[823]	validation_0-auc:0.76848
[824]	validation_0-auc:0.76837
[825]	validation_0-auc:0.76838
[826]	validation_0-auc:0.76836
[827]	validation_0-auc:0.76835
[828]	validation_0-auc:0.76837
[829]	va

[160]	validation_0-auc:0.72591
[161]	validation_0-auc:0.72556
[162]	validation_0-auc:0.72563
[163]	validation_0-auc:0.72598
[164]	validation_0-auc:0.72655
[165]	validation_0-auc:0.72686
[166]	validation_0-auc:0.72672
[167]	validation_0-auc:0.72610
[168]	validation_0-auc:0.72642
[169]	validation_0-auc:0.72620
[170]	validation_0-auc:0.72741
[171]	validation_0-auc:0.72812
[172]	validation_0-auc:0.72867
[173]	validation_0-auc:0.72922
[174]	validation_0-auc:0.72943
[175]	validation_0-auc:0.72915
[176]	validation_0-auc:0.72978
[177]	validation_0-auc:0.73001
[178]	validation_0-auc:0.72988
[179]	validation_0-auc:0.73009
[180]	validation_0-auc:0.72941
[181]	validation_0-auc:0.72959
[182]	validation_0-auc:0.72972
[183]	validation_0-auc:0.72903
[184]	validation_0-auc:0.72939
[185]	validation_0-auc:0.72924
[186]	validation_0-auc:0.72956
stop at 137, score=0.7302710996623011
mean_test_score=0.7559335259465456


UnboundLocalError: local variable 'best_param_booster' referenced before assignment